In [48]:
import requests
import json
import pandas as pd

url = "http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"

head = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}

payload = {"bld" : "dbms/MDC/STAT/standard/MDCSTAT01901",
"mktId" : "ALL",
"share" : "1",
"csvxls_isNo" : "fals",}

r = requests.post(url, headers=head, data=payload)

r.json()

stock_code = pd.DataFrame(r.json()['OutBlock_1'])
naver_url = "https://finance.naver.com/item/sise_day.nhn?code={}&page={}"

In [34]:
otp_key = r.text

In [35]:
target_url = "http://marketdata.krx.co.kr/contents/MKD/99/MKD99000001.jspx"

In [36]:
payload

{'bld': 'dbms/MDC/STAT/standard/MDCSTAT01901',
 'mktId': 'ALL',
 'share': '1',
 'csvxls_isNo': 'fals'}

In [37]:
r = requests.post(target_url, data=payload, headers=head)

In [38]:
r.text

'<html lang="ko">\r\n<head>\r\n  <meta charset="utf-8">\r\n  <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\r\n  <title>정보데이터시스템</title>\r\n  <script type="text/javascript" src="/WEB-APP/webponent/ci/agent.js"></script>\r\n</head>\r\n<body>\r\n<script>\r\n  if (ci.agent.isMobile) {\r\n    window.location.href=\'/contents/MMC/MAIN/main/index.cmd\';\r\n  } else {\r\n    window.location.href=\'/contents/MDC/MAIN/main/index.cmd\';\r\n  }\r\n</script>\r\n</body>\r\n</html>\r\n'

In [39]:
result_json = json.loads(r.text)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [20]:
type(result_json)

dict

In [9]:
result_json['block1']

KeyError: 'block1'

In [40]:
stock_code = pd.DataFrame(result_json['block1'])

KeyError: 'block1'

In [19]:
stock_code

,full_code,short_code,codeName,marketName
0,KR7060310000,A060310,3S,KOSDAQ
1,KR7095570008,A095570,AJ네트웍스,KOSPI
2,KR7006840003,A006840,AK홀딩스,KOSPI
3,KR7054620000,A054620,APS홀딩스,KOSDAQ
4,KR7265520007,A265520,AP시스템,KOSDAQ
...,...,...,...,...
2606,KR7000542001,A000547,흥국화재2우B,KOSPI
2607,KR7000541003,A000545,흥국화재우,KOSPI
2608,KR7003280005,A003280,흥아해운,KOSPI
2609,KR7037440005,A037440,희림,KOSDAQ


# naver 금융

In [49]:
naver_url = "https://finance.naver.com/item/sise_day.nhn?code={}&page={}"

In [53]:
r = requests.get(naver_url.format('060310', 1))

In [56]:
naver_url.format('060310', 1)

'https://finance.naver.com/item/sise_day.nhn?code=060310&page=1'

In [55]:
r.text

'\n<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">\n<html>\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=euc-kr">\n<title>네이버 :: 세상의 모든 지식, 네이버</title>\n\n<style type="text/css">\n.error_content * {margin:0;padding:0;}\n.error_content img{border:none;}\n.error_content em {font-style:normal;}\n.error_content {width:410px; margin:80px auto 0; padding:57px 0 0 0; font-size:12px; font-family:"나눔고딕", "NanumGothic", "돋움", Dotum, AppleGothic, Sans-serif; text-align:left; line-height:14px; background:url(https://ssl.pstatic.net/static/common/error/090610/bg_thumb.gif) no-repeat center top; white-space:nowrap;}\n.error_content p{margin:0;}\n.error_content .error_desc {margin-bottom:21px; overflow:hidden; text-align:center;}\n.error_content .error_desc2 {margin-bottom:11px; padding-bottom:7px; color:#888; line-height:18px; border-bottom:1px solid #eee;}\n.error_content .error_desc3 {clear:both; color:#888;}\n.error_con

In [54]:
pd.read_html(r.text)[0].dropna()

ValueError: No tables found

In [26]:
for row in stock_code[['short_code', 'codeName']].iterrows():
    code, name = row[1]
    total = []
    print(name)
    for count in range(1,16):
        r = requests.get(naver_url.format(code[1:], count))
        df = pd.read_html(r.text)[0].dropna()
        df['code'] = code
        df['name'] = name
        total.append(df)
    total_df = pd.concat(total)
    total_df.to_csv("./{}.csv".format(code), index=False, encoding='utf-8-sig')
      


NameError: name 'stock_code' is not defined